# Backtesting Demo

This notebook demonstrates the indicator and backtesting engine used in the Crypto IoT project.  It fetches historical OHLC data, computes Bollinger Bands and MACD, runs a simple strategy backtest, and surfaces concrete past examples where the combined rule triggered.  This notebook is reproducible and can be run inside the project container.

In [ ]:
# Install dependencies if needed (uncomment)
# !pip install pandas asyncio httpx ta matplotlib


In [ ]:
import asyncio
from datetime import datetime, timedelta
import pandas as pd
from services.indicator_engine_py.core import fetch_ohlc, compute_macd, compute_bollinger, macd_cross_signal, bollinger_signal, combine_signals, backtest_strategy, find_examples

# Define the symbol and time range
ticker = 'BTCUSDT'
interval = '4h'
end = datetime.utcnow()
start = end - timedelta(days=18*30)  # approx 18 months

async def prepare_data():
    df = await fetch_ohlc(ticker, interval, start, end)
    macd_df = compute_macd(df['close'])
    bb_df = compute_bollinger(df['close'])
    signal_macd = macd_cross_signal(macd_df['macd'], macd_df['macd_signal'])
    signal_bb = bollinger_signal(df['close'], bb_df['bb_lower'], bb_df['bb_upper'])
    combined = combine_signals([signal_macd, signal_bb], logic='and')
    return df, macd_df, bb_df, combined

# Run the preparation
loop = asyncio.get_event_loop()
df, macd_df, bb_df, signal_series = loop.run_until_complete(prepare_data())

print(df.tail())
print(signal_series.value_counts())


In [ ]:
# Backtest the combined strategy
result = backtest_strategy(df, signal_series, initial_capital=10000, fee_pct=0.0005)
print('Final PnL:', result.pnl)
print('Win rate:', result.win_rate)
print('Max drawdown:', result.max_drawdown)
print('Sharpe ratio:', result.sharpe)


In [ ]:
# Find concrete examples
examples = find_examples(
    df,
    signal_series,
    indicators={'macd': macd_df, 'bollinger': bb_df},
    num_examples=3,
    symbol='BTCUSDT',
    output_dir='analytics/outputs/examples'
)
examples


After running the notebook you will find PNG charts for each example in the `analytics/outputs/examples/` directory.  Each chart shows the price series together with Bollinger Bands and MACD lines, with a vertical magenta line marking the signal bar.

## Interpretation

The backtest metrics provide a high‑level view of the performance of the strategy.  You should interpret the PnL, win rate, max drawdown and Sharpe ratio in the context of your risk tolerance and trading style.  The example signals illustrate how the combination of Bollinger Bands and MACD crossovers can identify potential reversals or breakouts.